<a href="https://colab.research.google.com/github/h4rrye/3D_genome_organization/blob/main/chromosome_surface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center> <h1> <b> <i> Solvent Accessible Surface Area for Chromosome Structure </i> </b> </h1> </center>
<hr>
<br>

<h2> SCOPE OF THE PROJECT </h2>
<h4>
The project aims to develop an algorithm for calculating the Solvent Accessible Surface Area (SASA) of chromosomes. This algorithm generates a surface around the chromosome, representing the interface where genetic material interacts with intracellular molecules such as proteins, Transcription Factors and other DNA-altering compounds. It is designed to be adaptable, allowing for the integration of additional features in the future.

The primary objective of this algorithm is to identify accessible regions of the genome that are exposed to external influences, making them more vulnerable to mutations. These mutations often underlie major diseases like cancer. By studying this phenomenon, the project seeks to provide deeper insights into how 3D genome organization contributes to cancer susceptibility and how gene editing technologies might be leveraged to correct such vulnerabilities.
</h4>
<hr>
<h2> FUTURE SCOPE </h2>
<h4>
The future scope of this project includes quantifying the physical properties of the genomic structure and integrating functional data, such as gene expression, histone modifications, and protein binding sites. By mapping these features onto the genomic structure, the project aims to uncover how physical properties influence genomic function and susceptibility. Additionally, this framework can be extended to study changes in the genomic architecture of cancerous cells, identifying underlying structural alterations alongside shifts in expression levels. This comprehensive approach will provide valuable insights into disease mechanisms, contributing to a deeper understanding of how structural and functional dynamics drive genomic vulnerabilities and potential therapeutic strategies.
<br>
<hr>
<h3> <b> Data Used </b> </h3>
<h4>
The input data for this script is a PDB file for a chromosome structure, from <b> GSDB </b> : Genome Structure DataBase

<hr>




<h2> Hyperparameters </h2>
the following parameters are used to optimize the surface around the chromosome, along with the values used for the particular example.
<br> <br>
<h3>
1. <font color='red'><b><i>oc</i></b></font>  : overlapping coefficient : <b>1.45</b> <br>
2. <font color='red'><b><i>r</i></b></font> : oc * max eucledian distance between points. <br>
4. <font color='red'><b><i>d</i></b></font> : density of points in the shpere per axis : <b>15</b> <br>
3. <font color='red'><b><i>b</i></b></font> : number of bins per axis in the box : <b>50</b> <br>
</h3>
<br>
<hr>
<br>

In [ ]:
# ================================
#           libraries
# ================================

# %pip install -q MDTraj    # leave this uncommented if running it on Google Colab
import mdtraj as md

import pandas as pd
import numpy as np
import scipy.ndimage as nd
import math
from tqdm import tqdm
from random import random
import plotly.graph_objects as go
import plotly.io as pio

import urllib.request
import ssl
from copy import deepcopy


In [ ]:

# import pdb

# the structure used in this script is human chromosome 1 at 500kbp resolution
# data is from GSDB: Genome Structure DataBase

"""
# ==========================================================
# this used to work but not anymore, the block below works
# ==========================================================

url = 'https://calla.rnet.missouri.edu/genome3d/GSDB/Database/AX9716PF/GSE105544_ENCFF010WBP/VC/LorDG/chr1.pdb'

chr_str = md.load_pdb(url)
coords = chr_str.xyz.reshape(-1, 3)
coords_df = pd.DataFrame(coords, columns=['x', 'y', 'z'])
coords_df.head(10)
# ==========================================================
"""

"\n# ==========================================================\n# this used to work but not anymore, the block below works\n# ==========================================================\n\nurl = 'https://calla.rnet.missouri.edu/genome3d/GSDB/Database/AX9716PF/GSE105544_ENCFF010WBP/VC/LorDG/chr1.pdb'\n\nchr_str = md.load_pdb(url)\ncoords = chr_str.xyz.reshape(-1, 3)\ncoords_df = pd.DataFrame(coords, columns=['x', 'y', 'z'])\ncoords_df.head(10)\n# ==========================================================\n"

In [ ]:

# Create unverified context for this specific download
ssl_context = ssl._create_unverified_context()

url = 'https://calla.rnet.missouri.edu/genome3d/GSDB/Database/AX9716PF/GSE105544_ENCFF010WBP/VC/LorDG/chr1.pdb'

# Download the file
with urllib.request.urlopen(url, context=ssl_context) as response:
    with open('chr1.pdb', 'wb') as out_file:
        out_file.write(response.read())

# Load from local file
chr_str = md.load_pdb('chr1.pdb')
coords = chr_str.xyz.reshape(-1, 3)



In [24]:
coords

array([[-0.3264,  0.3988, -0.265 ],
       [-0.3026,  0.4367, -0.2681],
       [-0.2171,  0.5026, -0.2616],
       ...,
       [-0.3446,  0.1994,  0.5161],
       [-0.3101,  0.203 ,  0.5102],
       [-0.3066,  0.2338,  0.4595]], shape=(451, 3), dtype=float32)

In [25]:

# plotting an interactive 3D line plot of the chromosome structure


line_dict = dict(width=5, color=coords[:,2], colorscale='Viridis')
title_dict = dict(text='3D Chromosome Structure', font=dict(size=24), x=0.5)
data = [go.Scatter3d( 
            x=coords[:,0], 
            y=coords[:,1], 
            z=coords[:,2], 
            mode='lines', 
            line=line_dict
)]
fig = go.Figure(data)
fig.update_layout(width=800, height=800, title=title_dict)
fig.show()
# fig.write_html("chr_model.html")
# pio.write_image(fig, "chr_model.png", format="png", width=800, height=800, scale=1)

[View the 3D Chromosome Plotly Visualization](plots/chr_model.png)

In [26]:

# create overlapping spheres around every coordinate to envelop the entire chromosome
# calculate consecutive euclidian distances to calculate the radius of the spheres.
# (radius = max euclidian distance * overlapping factor) to ensure overlap between all the spheres.
#_______________________________________________________________________________________________________


# calculate distances
class euc_dist:
    def __init__(self, df):
        self.df = df
        self.dists = []
        self.calulate_dist()


    def calulate_dist(self):
        for i in range(len(self.df) - 1):
            d = math.dist(self.df[i, :], self.df[i+1, :])         # >> math.dist similar to np.linalg.norm(ord=2)
            self.dists.append(d)


    def avg_dist(self):
        return np.mean(self.dists)

    def max_dist(self):
        return max(self.dists)


dd = euc_dist(coords)


oc = 1.45                           # >> overlapping coefficient
r = oc * dd.max_dist()              # >> radius of the sphere
d = 15                             # >> density of dots in the sphere

# _______________________________________________________________________________________
# create spheres around the points

def create_spheres(x, y, z, r=r, d=d):

    rad = np.linspace(0, r, int(d/2))
    phi = np.linspace(0, np.pi, d)
    theta = np.linspace(0, 2*np.pi, d, endpoint=False)

    theta, phi, rad = np.meshgrid(theta, phi, rad, indexing='ij')

    phi = phi.flatten()
    theta = theta.flatten()
    rad = rad.flatten()

    dx = rad * np.sin(phi) * np.cos(theta) + x
    dy = rad * np.sin(phi) * np.sin(theta) + y
    dz = rad * np.cos(phi) + z

    spheres = np.array([dx, dy, dz])

    return spheres


In [27]:

all_spheres = np.empty((3,0))

for rows in tqdm(coords, total=coords.shape[0]):
    spheres = create_spheres(rows[0], rows[1], rows[2], r, d)
    all_spheres = np.hstack([all_spheres, spheres])

all_spheres = np.array(all_spheres).T
all_spheres.shape

# scaling the values to +ve
scaling_factor = abs(np.min(all_spheres) * 1.25)
all_spheres += scaling_factor

coords += scaling_factor


'''
# visualize the sphere-filled structure
marker_dict = dict(size=4, opacity=0.1, color='grey')
title_dict = dict(text='spheres around the chromosome', x=0.5, font=dict(size=24))
data = go.Scatter3d(x=all_spheres[:,0], y=all_spheres[:,1], z=all_spheres[:,2], mode='markers', marker=marker_dict)
fig = go.Figure(data)
fig.update_layout(width=800, height=800, title=title_dict)
fig.show()
'''


100%|██████████| 451/451 [00:02<00:00, 165.74it/s]


"\n# visualize the sphere-filled structure\nmarker_dict = dict(size=4, opacity=0.1, color='grey')\ntitle_dict = dict(text='spheres around the chromosome', x=0.5, font=dict(size=24))\ndata = go.Scatter3d(x=all_spheres[:,0], y=all_spheres[:,1], z=all_spheres[:,2], mode='markers', marker=marker_dict)\nfig = go.Figure(data)\nfig.update_layout(width=800, height=800, title=title_dict)\nfig.show()\n"

In [45]:
coords - scaling_factor

array([[-0.32640003,  0.39880003, -0.26499997],
       [-0.30260001,  0.4367    , -0.26810001],
       [-0.21710001,  0.50259997, -0.2616    ],
       ...,
       [-0.34460001,  0.19939996,  0.51609994],
       [-0.31009995,  0.20299996,  0.51020004],
       [-0.30659996,  0.23379995,  0.45949997]], shape=(451, 3))

In [29]:

# create a cube and place the structure in it
# the cube is further divided into multiple bins = b


x_lim = [min(all_spheres[:,0]), max(all_spheres[:,0])]
y_lim = [min(all_spheres[:,1]), max(all_spheres[:,1])]
z_lim = [min(all_spheres[:,2]), max(all_spheres[:,2])]

edge_size = max((x_lim[1] - x_lim[0]), (y_lim[1] - y_lim[0]), (z_lim[1] - z_lim[0]))
padding = edge_size * 0.1
edge_size += padding

b = 50         # >> number of bins per side

bin_size = edge_size / b

box = np.zeros((b, b, b), dtype='float')

for row in all_spheres:
    x, y, z = row
    x_index = int(x // bin_size)
    y_index = int(y // bin_size)
    z_index = int(z // bin_size)

    if (0 <= x_index < b) and (0 <= y_index < b) and (0 <= z_index < b):
        box[x_index, y_index, z_index] = 1



In [30]:
coords

array([[1.0031081 , 1.7283082 , 1.0645082 ],
       [1.0269082 , 1.7662082 , 1.0614082 ],
       [1.1124082 , 1.8321081 , 1.0679082 ],
       ...,
       [0.98490816, 1.5289081 , 1.8456081 ],
       [1.0194082 , 1.5325081 , 1.8397082 ],
       [1.0229082 , 1.5633081 , 1.7890081 ]],
      shape=(451, 3), dtype=float32)

In [31]:

# find the surface bins

# create a box to move around the entire cube
kernel = np.ones ((3, 3, 3), dtype='int')
kernel[1, 1, 1] = 0

neighbour_count = nd.convolve(box, kernel, mode='constant', cval=0)

surface_bins = (box == 1) & (neighbour_count < 26)

surface_coords = np.argwhere(surface_bins == 1)         # >> indices of bin value = 1


In [32]:
coords

array([[1.0031081 , 1.7283082 , 1.0645082 ],
       [1.0269082 , 1.7662082 , 1.0614082 ],
       [1.1124082 , 1.8321081 , 1.0679082 ],
       ...,
       [0.98490816, 1.5289081 , 1.8456081 ],
       [1.0194082 , 1.5325081 , 1.8397082 ],
       [1.0229082 , 1.5633081 , 1.7890081 ]],
      shape=(451, 3), dtype=float32)

In [33]:

# 3D visualization using plotly graph_objects
    # line plot for the chromosome
    # diffused dots/cloud for the surface
    # use color to map various parameters

##############################################################################################################################################################

marker_dict = dict(size=5, opacity=0.1, color='navy')
line_dict = dict(width=5, color='red')
title_dict = dict(text='SASA for Chromosome', x=0.5, font=dict(size=24))
data = go.Scatter3d(
    x=surface_coords[:,0]*bin_size, 
    y=surface_coords[:,1]*bin_size, 
    z=surface_coords[:,2]*bin_size, 
    mode='markers', 
    marker=marker_dict, 
    name='surface'
)
fig = go.Figure(data)
fig.add_trace(go.Scatter3d( 
                    x=coords[:, 0], 
                    y=coords[:, 1], 
                    z=coords[:, 2], 
                    mode='lines', 
                    line=line_dict, 
                    name='chromosome'
))
fig.update_layout(
    scene = dict(
        xaxis = dict(visible=False),
        yaxis = dict(visible=False),
        zaxis = dict(visible=False)
    ),
    margin=dict(l=0, r=0, b=0, t=0, pad=0),
    paper_bgcolor='rgba(0,0,0,0)',
    showlegend=False,
    height=800, 
    width=800, 
    title=title_dict
)
fig.show()
# fig.write_html("sasa_model.html")
# fig.write_image("sasa_model.jpg")


[View the SASA model plotly visualization](plots/sasa_model.png)

<h1>
<b>
<div align="center"> `END` </div>
</b>
</h1>

<h1> <font color='red'> <b><i>TO DO</b></i> </font> </h1>
<h2>
1. a dashboard with crhromsome structure and various plots for the calculated physical attributes.<br>
2. create <b><u>class</u></b> and define <b><u>func</u></b> for all the processes <br>
3. quantify various physical attributes using scipy and other scientific packages.<br>
4. create a simple Nextflow pipeline, stitching:<br>
        - `surface model`<br>
        - `calculate and map physical attributes` <br>
        - `map biological features`<br>
5. create a docker image running the entire pipeline<br>
6. deploy the entire dashboard and pipeline on AWS Batch<br>

<br>
<hr>


In [41]:
# coords, surface_coords

def eucledian_distance_3d(c1, c2) -> float:
    return np.sqrt(sum(np.array(c1), np.array(c2)))


surface_coords_scaled = surface_coords * bin_size

coord_dist = []
for coord in coords:
    distances = np.linalg.norm(surface_coords_scaled - coord, axis=1)   # euc dist for all
    min_dist = np.min(distances)
    coord_dist.append([*coord, min_dist])   # unpack array using *

coord_dist = np.array(coord_dist)
coord_dist.shape


(451, 4)

In [42]:
coord_dist

array([[1.00310814, 1.7283082 , 1.0645082 , 0.09739255],
       [1.02690816, 1.76620817, 1.06140816, 0.14089538],
       [1.11240816, 1.83210814, 1.06790817, 0.17768105],
       ...,
       [0.98490816, 1.52890813, 1.84560812, 0.32730337],
       [1.01940823, 1.53250813, 1.83970821, 0.35112396],
       [1.02290821, 1.56330812, 1.78900814, 0.29451623]], shape=(451, 4))

In [ ]:
np.save('../data/processed/x_y_z_dist_surf.npy', coord_dist)

In [39]:
surface_coords * bin_size

array([[0.48965688, 1.6158677 , 0.78345101],
       [0.48965688, 1.71379908, 0.78345101],
       [0.53862257, 1.32207357, 1.95862752],
       ...,
       [2.25242164, 1.02827945, 0.63655394],
       [2.25242164, 1.02827945, 0.68551963],
       [2.25242164, 1.02827945, 0.78345101]], shape=(8325, 3))

In [ ]:
surface_coords_scaled = surface_coords * bin_size - scaling_factor
np.save('../data/processed/surface_coords_scaled.npy', surface_coords_scaled)
